In [1]:
"""
Automatic speech recognition and speaker identity
"""
from __future__ import division, print_function, absolute_import
import os
import numpy as np
import sklearn.preprocessing as prep
import librosa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses

In [2]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=10):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    print("file read=",file_path)
    print(wave.shape,sr)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sr).transpose()
    #print(mfcc.shape)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    '''
    classdocs
    '''
    def __init__(self, filePath="/home/francisco/voz"):
        '''
        Constructor
        '''
        basepath=filePath
        self.mfccs=[]
        self.speakers=[]
        self.words=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n))
            self.mfccs.append(mfccs)
            #decode file name format speaker_word_number
            speaker,word,number=n.split("_")
            self.speakers.append(speaker)
            self.words.append(word)
        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.leWords=prep.LabelEncoder()
        self.leWords.fit(self.words)
        self.iWordLabels=self.leWords.transform(self.words)
        print(len(self.words))
        print(self.words)
        print(self.speakers)

ea=EasyASR()
print(ea.leWords)

Imports doned
fNames= 108
file read= /home/francisco/voz/paco_no_006.wav
(3871,) 16000
file read= /home/francisco/voz/paco_si_008.wav
(4963,) 16000
file read= /home/francisco/voz/paco_dos_002.wav
(9285,) 16000
file read= /home/francisco/voz/paco_uno_004.wav
(4434,) 16000
file read= /home/francisco/voz/paco_no_007.wav
(2428,) 16000
file read= /home/francisco/voz/paco_para_003.wav
(4882,) 16000
file read= /home/francisco/voz/paco_cuatro_003.wav
(6972,) 16000
file read= /home/francisco/voz/paco_nueve_001.wav
(6524,) 16000
file read= /home/francisco/voz/carlos_si_002.wav
(4630,) 16000
file read= /home/francisco/voz/paco_buenos-dias_001.wav
(14450,) 16000
file read= /home/francisco/voz/paco_buenos-dias_003.wav
(14816,) 16000
file read= /home/francisco/voz/irene_hola_003.wav
(6847,) 16000
file read= /home/francisco/voz/paco_seis_001.wav
(9176,) 16000
file read= /home/francisco/voz/paco_buenos-dias_007.wav
(8618,) 16000
file read= /home/francisco/voz/paco_ocho_001.wav
(6462,) 16000
file read=

In [74]:
trainX, trainYwords, trainYspeakers = np.array(ea.mfccs),np.array(ea.iWordLabels),np.array(ea.iSpeakerLabels)
#testX, testY = test

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputWords_dim=max(trainYwords)+1
outputSpeakers_dim=max(trainYspeakers)+1

print(outputWords_dim,outputSpeakers_dim)

# Converting labels to binary vectors
trainYwords_cat    = utils.to_categorical(trainYwords   ,outputWords_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
print (trainX.shape,trainYwords_cat.shape,trainYspeakers_cat.shape)

17 4
(108, 10, 20) (108, 17) (108, 4)


In [77]:
# Network building
def build_model_asr_conv1d(input_dim,outputWords_dim,outputSpeakers_dim,hiden_dim=128):
    inputs = tf.keras.Input(shape=(None,input_dim))

    # Conv1D + global max pooling
    x = layers.Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(inputs)
    #x = layers.Dropout(0.5)(x)
    x = layers.Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    #x = layers.Dropout(0.5)(x)
    #x = layers.Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = layers.GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    x = layers.Dense(hiden_dim, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_words    = layers.Dense(outputWords_dim   , activation="softmax", name="prediction_words")(x)
    prediction_speakers = layers.Dense(outputSpeakers_dim, activation="softmax", name="prediction_speakers")(x)

    model = tf.keras.Model(inputs, [prediction_words,prediction_speakers])
    return model

def build_model_asr_lstm(input_dim,outputs,hiden_dim=128):
    input = Input(shape=(None,maxlen,input_dim))
    x,h,c = LSTM(hiden_dim, input_dim=input_dim, input_length=maxlen)(input)
    #x = Bidirectional()(x)
    x = Dropout(0.65)(x)
    x = Dense(outputs,activation="softmax")(x)
    return Model(input, x)

model=build_model_asr_conv1d(input_dim,outputWords_dim,outputSpeakers_dim)
#model=build_model_asr_conv1d(input_dim,outputWords_dim,outputSpeakers_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, None, 20)]   0                                            
__________________________________________________________________________________________________
conv1d_23 (Conv1D)              (None, None, 128)    7808        input_19[0][0]                   
__________________________________________________________________________________________________
conv1d_24 (Conv1D)              (None, None, 128)    49280       conv1d_23[0][0]                  
__________________________________________________________________________________________________
global_max_pooling1d_10 (Global (None, 128)          0           conv1d_24[0][0]                  
___________________________________________________________________________________________

In [78]:
# Training
history=model.fit(trainX, [trainYwords_cat,trainYspeakers_cat], batch_size=2,nb_epoch=50,validation_split=0.2)

Train on 108 samples
Epoch 1/50
108/108 [==============================] - 1s 6ms/sample - loss: 16.9806 - prediction_words_loss: 13.1862 - prediction_speakers_loss: 3.7944 - prediction_words_accuracy: 0.0926 - prediction_speakers_accuracy: 0.5093  
Epoch 2/50
108/108 [==============================] - 0s 1ms/sample - loss: 3.8094 - prediction_words_loss: 2.9104 - prediction_speakers_loss: 0.8989 - prediction_words_accuracy: 0.2685 - prediction_speakers_accuracy: 0.7685
Epoch 3/50
108/108 [==============================] - 0s 1ms/sample - loss: 2.9663 - prediction_words_loss: 2.1161 - prediction_speakers_loss: 0.8502 - prediction_words_accuracy: 0.3611 - prediction_speakers_accuracy: 0.7593
Epoch 4/50
108/108 [==============================] - 0s 1ms/sample - loss: 2.9908 - prediction_words_loss: 2.3111 - prediction_speakers_loss: 0.6796 - prediction_words_accuracy: 0.3611 - prediction_speakers_accuracy: 0.7407
Epoch 5/50
108/108 [==============================] - 0s 1ms/sample - loss:

In [89]:
print(trainX[5:7,:,:].shape)
predWord,predSpeaker=model.predict(trainX[5:6,:,:])
print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_)

(2, 10, 20)
[[1.8516155e-03 2.8807502e-03 1.4103903e-05 8.0832740e-07 6.3417517e-08
  2.1912331e-04 3.1062678e-05 8.2082192e-07 9.5103615e-06 3.1980427e-08
  9.9484324e-01 7.8451530e-06 4.0716599e-09 4.6922705e-07 4.5425601e-09
  1.4017861e-04 3.0485876e-07]] [[1.5889159e-09 2.9108355e-10 2.2578655e-09 1.0000000e+00]]
10 = 0.99484324
3 = 1.0
paco  ha dicho ' para '
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
['carlos' 'eva' 'irene' 'paco']
